In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

In [1]:
# Load Packages
import tensorflow
from tensorflow.keras import backend
#from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from keras.models import load_model, save_model
import numpy as np
import pandas as pd
import random
import sys
import io
import re
import os

Using TensorFlow backend.


In [2]:
os.chdir('/content/drive/My Drive/Colab Notebooks/NLP Group Project/')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/My Drive/Colab Notebooks/NLP Group Project/'

In [4]:
# Read Songs
songs = pd.read_csv('drake-songs.csv')

In [5]:
text = ''

for index, row in songs['lyrics'].iteritems():
    cleaned = str(row).lower().replace(' ', '\n')
    text = text + " ".join(re.findall(r"[a-z']+", cleaned))
    
len(text)

367372

In [6]:
text[:1000]

"money money cars cars clothes clothes the hoes i suppose yeah i want the money money and the cars cars and the clothes the hoes i suppose i just wanna be i just wanna be successful i just wanna be i just wanna be successful i just wanna be i just wanna be successful drizzy ah yeah trey i fuckin' feel you they be starin' at the money like it's unfamiliar i get it i live it to me there's nothings realer just enough to solve your problems too much will kill ya and when i leave i always come right back here the young spit'a that everybody in rap fear a lot of y'all are still soundin' like last year the game needs change and i'm the mofucking cashier nickels for my thoughts dimes in my bed quarters of the kush shape the lines in my head take my verses too serious ya hate me 'cause i'm the one to paint a vivid picture no hd yeah i want it all that's why i strive for it dis me and you'll never hear a reply for it any awards show or party i get fly for it i know that it's comin' i just hope t

In [7]:
tokens = text.replace("'", '').split(' ')

chars = sorted(list(set(tokens)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

del char_indices['']
del indices_char[0]

total chars: 5865


In [8]:
tokens[:1000]

['money',
 'money',
 'cars',
 'cars',
 'clothes',
 'clothes',
 'the',
 'hoes',
 'i',
 'suppose',
 'yeah',
 'i',
 'want',
 'the',
 'money',
 'money',
 'and',
 'the',
 'cars',
 'cars',
 'and',
 'the',
 'clothes',
 'the',
 'hoes',
 'i',
 'suppose',
 'i',
 'just',
 'wanna',
 'be',
 'i',
 'just',
 'wanna',
 'be',
 'successful',
 'i',
 'just',
 'wanna',
 'be',
 'i',
 'just',
 'wanna',
 'be',
 'successful',
 'i',
 'just',
 'wanna',
 'be',
 'i',
 'just',
 'wanna',
 'be',
 'successful',
 'drizzy',
 'ah',
 'yeah',
 'trey',
 'i',
 'fuckin',
 'feel',
 'you',
 'they',
 'be',
 'starin',
 'at',
 'the',
 'money',
 'like',
 'its',
 'unfamiliar',
 'i',
 'get',
 'it',
 'i',
 'live',
 'it',
 'to',
 'me',
 'theres',
 'nothings',
 'realer',
 'just',
 'enough',
 'to',
 'solve',
 'your',
 'problems',
 'too',
 'much',
 'will',
 'kill',
 'ya',
 'and',
 'when',
 'i',
 'leave',
 'i',
 'always',
 'come',
 'right',
 'back',
 'here',
 'the',
 'young',
 'spita',
 'that',
 'everybody',
 'in',
 'rap',
 'fear',
 'a',
 '

In [9]:
maxlen = 15
sentences = []
next_chars = []
for i in range(0, len(tokens)-maxlen-5, 1):
    sentences.append(tokens[i:i+maxlen])
    next_chars.append(tokens[i+maxlen])

In [10]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    y[i, char_indices[next_chars[i]]] = 1
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1

KeyError: ''

In [11]:
# check CuDNNLSTM on VM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 15, 128)           3068928   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 5865)              756585    
_________________________________________________________________
activation (Activation)      (None, 5865)              0         
Total params: 3,957,097
Trainable params: 3,957,097
Non-trainable params: 0
_________________________________________________________________


In [13]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds[0], 5865)
    return np.argmax(probas)

In [14]:
def on_epoch_end(epoch, logs):
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(tokens) - maxlen - 1)
    for diversity in [0.2, 0.5, 0.8]:
        print('----- diversity:', diversity)
        
        generated = ''
        sentence = tokens[start_index: start_index + maxlen]
        for i in sentence:
            generated += str(i)
            generated += ' '

        sentence_list = ''
        for i in sentence:
            sentence_list += str(i) + ' '
        print('----- Generating with seed: "' + sentence_list + '"')
        sys.stdout.write(generated)

        for i in range(25):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence.append(next_char)
            sentence.pop(0)

            sys.stdout.write(next_char)
            sys.stdout.write(' ')

            sys.stdout.flush()
            
        print()

In [36]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

history = model.fit(
    x, 
    y,
    batch_size=512,
    epochs=15,
    callbacks=[print_callback]
)

Train on 75057 samples
Epoch 1/15
74752/75057 [============================>.] - ETA: 0s - loss: 1.6696----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "the champagne your best friend is bar tending your parents sayin this another phase in "
the champagne your best friend is bar tending your parents sayin this another phase in i im i i the i the the i i im the i on the you i the the i the i i i you 
----- diversity: 0.5
----- Generating with seed: "the champagne your best friend is bar tending your parents sayin this another phase in "
the champagne your best friend is bar tending your parents sayin this another phase in so me couple girl i im a to soon too a how the i me you me pillow a i time like make following the 
----- diversity: 0.8
----- Generating with seed: "the champagne your best friend is bar tending your parents sayin this another phase in "
the champagne your best friend is bar tending your parents sayin this another phase in taste je

too high to be friendly they throw dirt on my name well thats why they had maybe spilling yeah time dont what best i what i be im and want had a you what that best bone before shorty fragrance 
75057/75057 [==============================] - 232s 3ms/sample - loss: 1.5666


In [37]:
model.save('drake_word_model.h5')

In [15]:
def generate_text(model, seed, length, diversity):

    maxlen = 15
    generated = ''
    sentence = tokens[seed: seed + maxlen]
    
    for i in sentence:
        generated += str(i)
        generated += ' '

    sentence_list = ''
    for i in sentence:
        sentence_list += str(i) + ' '
    print('----- Generating with seed: "' + sentence_list + '"')
    
    sys.stdout.write(generated)

    for i in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence.append(next_char)
        sentence.pop(0)

        sys.stdout.write(next_char)
        sys.stdout.write(' ')

        sys.stdout.flush()

    print()

In [16]:
generate_text(model, 1234, 10, 0.5)

----- Generating with seed: "fuckin best you the best i ever had best i ever had best i ever "
fuckin best you the best i ever had best i ever had best i ever 5231
today 4143
report 941
clearing 3192
minutes 2049
georgia 1509
drought 1249
days 718
c 2705
kept 4249
roper 
